In [2]:
import os
import xlsxwriter

In [3]:
from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel
from clarifai_grpc.grpc.api import service_pb2_grpc
from clarifai_grpc.grpc.api import service_pb2, resources_pb2
from clarifai_grpc.grpc.api.status import status_code_pb2

stub = service_pb2_grpc.V2Stub(ClarifaiChannel.get_grpc_channel())

In [4]:
# Send image to Clarifai api, return generated tags
# Copied from the Clarif python demo: https://docs.clarifai.com/api-guide/predict/images

def generate_tags(img):
    # This is how you authenticate.
    metadata = (('authorization', 'Key 55264bb600de467ca0064ee35546481b'),)
    with open(img, "rb") as f:
        file_bytes = f.read()

    post_model_outputs_response = stub.PostModelOutputs(
        service_pb2.PostModelOutputsRequest(
            model_id="aaa03c23b3724a16a56b629203edc62c",
            #version_id="{THE_MODEL_VERSION_ID}",  # This is optional. Defaults to the latest model version.
            inputs=[
                resources_pb2.Input(
                    data=resources_pb2.Data(
                        image=resources_pb2.Image(
                            base64=file_bytes
                        )
                    )
                )
            ]
        ),
        metadata=metadata
    )

    if post_model_outputs_response.status.code != status_code_pb2.SUCCESS:
        raise Exception("Post model outputs failed, status: " + post_model_outputs_response.status.description)

    # Since we have one input, one output will exist here.
    output = post_model_outputs_response.outputs[0]
    return ["%s %.2f" % (concept.name, concept.value) for concept in output.data.concepts]


Exception: Post model outputs failed, status: Failure

In [17]:
# Generate tags and store them in excel
# Theme, StudentID, imgID, tags

rootdir = '/home/tim/Documents/scriptie/data'
workbook = xlsxwriter.Workbook('complete_dataset.xlsx')
worksheet = workbook.add_worksheet()
[worksheet.write(0, i, header) for i,header in 
                                enumerate(['PhotoID', 'Photopath', 'studentID', 'Theme', 'Tags[..]'])]
row = 1
for i in range(1,22):
    student = i
    studentdir = rootdir + '/' + str(i)
    for subdir, dirs, files in os.walk(studentdir):
        if subdir is not studentdir:
            for file in files:
                theme = os.path.basename(subdir)
                img_path = subdir + '/' + str(file)
                img_id = file.split(".")[0]
                tags = generate_tags(img_path)
                entry = [img_id, img_path, str(i), theme] + tags
                column = 0
                for value in entry:
                    worksheet.write(row, column, value)
                    worksheet.set_column(0,column,25)
                    column += 1
                row += 1
                
workbook.close()

574
